In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
from datetime import datetime

In [2]:
hot_df = pd.read_csv("./datasets/hotspot_historic.csv")
clim_df = pd.read_csv("./datasets/climate_historic.csv")

hot_df["date"] = pd.to_datetime(hot_df["date"])
clim_df["date"] = pd.to_datetime(clim_df["date"])

In [3]:
hot_df.describe()

,latitude,longitude,confidence,surface_temperature_celcius
count,2668.000000,2668.000000,2668.000000,2668.000000
mean,-36.889822,143.846864,77.061094,55.643178
std,0.808740,2.118841,13.159445,16.316451
min,-38.912000,140.967800,50.000000,28.000000
25%,-37.578850,142.211825,68.000000,44.000000
50%,-36.827750,143.365100,77.000000,51.000000
75%,-36.349300,145.220225,86.000000,61.000000
max,-34.243200,149.408800,100.000000,124.000000


In [4]:
clim_df.describe(include="datetime")

,date
count,366
unique,366
top,2017-10-30 00:00:00
freq,1
first,2016-12-31 00:00:00
last,2018-01-01 00:00:00


In [5]:
hot_df.describe(include="datetime")

,date
count,2668
unique,145
top,2017-04-13 00:00:00
freq,357
first,2017-01-04 00:00:00
last,2017-12-27 00:00:00


In [6]:
clim_df.head()

,station,date,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation
0,948700,2016-12-31,19,56.8,7.9,11.1,0.00I
1,948700,2017-02-01,15,50.7,9.2,13.0,0.02G
2,948700,2017-03-01,16,53.6,8.1,15.0,0.00G
3,948700,2017-04-01,24,61.6,7.7,14.0,0.00I
4,948700,2017-05-01,24,62.3,7.0,13.0,0.00I


# taskA

In [7]:
def taskA():
    client = MongoClient()

    # clean up db 
    client.drop_database("fit5148_assignment_db")

    # connect to db and if not exits, create it.
    db = client.fit5148_assignment_db

    # connect to collection and if not exits, create it.
    hotspot = db.hotspot
    climate = db.climate

    # set the field "date" as the unique index.
    climate.create_index([('date', pymongo.ASCENDING)],unique=True)

    # read csv into mongodb.

    data_hot = hot_df.to_dict(orient="records")
    hotspot.insert_many(data_hot)

    data_cli = clim_df.to_dict(orient="records")
    climate.insert_many(data_cli)

    # join two collections
    documents = climate.aggregate([{
    "$lookup":
        {   "localField": "date",
            "from": "hotspot",
            "foreignField" : "date",
            "as": "hotspot"
        }
    }])

    # get the results into db.document
    collection = db.documents
    collection.insert_many(documents)
    
    # drop temp collections
    db.hotspot.drop()
    db.climate.drop()
    
    client.close()
    # insert successfully
    return 0

In [8]:
taskA()

0

# task B

In [9]:
client = MongoClient()

# connect to db
db = client.fit5148_assignment_db

collection = db.documents

In [10]:
# one document
pprint(collection.find_one({}))

{'_id': ObjectId('5ccfe268c8b0712e2050b6d5'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2016, 12, 31, 0, 0),
 'hotspot': [],
 'max_wind_speed': 11.1,
 'precipitation ': ' 0.00I',
 'relative_humidity': 56.8,
 'station': 948700,
 'windspeed_knots': 7.9}


> **a**. Find climate data on ​10th December 2017​.

In [11]:
results = collection.find_one({"date":datetime(2017, 12, 10)},{"hotspot":0, "_id":0})
pprint(results)

{'air_temperature_celcius': 15,
 'date': datetime.datetime(2017, 12, 10, 0, 0),
 'max_wind_speed': 14.0,
 'precipitation ': ' 0.00G',
 'relative_humidity': 44.5,
 'station': 948702,
 'windspeed_knots': 10.2}


> **b**. Find the latitude​, ​longitude, surface temperature (​ °C)​, and ​confidence ​when the surface temperature (°C) was between ​65 °C​ and​ 100 °C​.

In [12]:
results = collection.aggregate([
    { "$match": { 
        "hotspot":{
            "$elemMatch":{"surface_temperature_celcius":{"$gte": 65,"$lte":100}}
        }
    }},
    {"$unwind":"$hotspot"},
    {"$match": { "hotspot.surface_temperature_celcius":{"$gte": 65,"$lte":100}}},
    {"$project":{"_id":0,"hotspot.latitude":1,"hotspot.longitude":1,
                 "hotspot.confidence":1,"hotspot.surface_temperature_celcius":1}}
    
])
i = 0
for doc in results:
    i+=1
    print(doc["hotspot"])

{'latitude': -37.2284, 'longitude': 147.9187, 'confidence': 94, 'surface_temperature_celcius': 73}
{'latitude': -37.6572, 'longitude': 142.0703, 'confidence': 97, 'surface_temperature_celcius': 80}
{'latitude': -37.0193, 'longitude': 148.1459, 'confidence': 84, 'surface_temperature_celcius': 71}
{'latitude': -37.4229, 'longitude': 147.02700000000002, 'confidence': 100, 'surface_temperature_celcius': 99}
{'latitude': -37.0055, 'longitude': 148.1582, 'confidence': 80, 'surface_temperature_celcius': 68}
{'latitude': -37.4128, 'longitude': 147.0242, 'confidence': 85, 'surface_temperature_celcius': 98}
{'latitude': -34.357, 'longitude': 141.5361, 'confidence': 90, 'surface_temperature_celcius': 67}
{'latitude': -34.3539, 'longitude': 141.5547, 'confidence': 93, 'surface_temperature_celcius': 72}
{'latitude': -36.9939, 'longitude': 148.2244, 'confidence': 90, 'surface_temperature_celcius': 68}
{'latitude': -36.9959, 'longitude': 148.2118, 'confidence': 95, 'surface_temperature_celcius': 75}


In [13]:
print("records found:", i)

hot_df[(65<=hot_df.surface_temperature_celcius) & (hot_df.surface_temperature_celcius<=100)].shape[0]

records found: 476


476

> **c**. Find ​date,​ ​surface temperature (°C), air temperature (°C), relative humidity and max wind speed on​ 15th and 16th of December 2016.

In [14]:
results = collection.find({"date":{"$gte":datetime(2016, 12, 15),"$lte":datetime(2016, 12, 17)}})
for doc in results:
    pprint(doc)

> **d**. Find ​datetime, air temperature (°C), surface temperature (°C) and ​confidence when the ​confidence is between 80 and 100.

In [15]:
results = collection.aggregate([
    { "$match": { 
        "hotspot":{
            "$elemMatch":{"confidence":{"$gte": 80,"$lte":100}}
        }
    }},
    {"$unwind":"$hotspot"},
    {"$match": { "hotspot.confidence":{"$gte": 80,"$lte":100}}},
    {"$project":{"_id":0, "air_temperature_celcius":1,
                 "hotspot.datetime":1,"hotspot.surface_temperature_celcius":1,
                 "hotspot.confidence":1}}
    
])
i = 0
for doc in results:
    i+=1
    print(doc)

{'air_temperature_celcius': 20, 'hotspot': {'datetime': '2017-03-06T05:06:30', 'confidence': 87, 'surface_temperature_celcius': 62}}
{'air_temperature_celcius': 20, 'hotspot': {'datetime': '2017-03-06T05:06:20', 'confidence': 85, 'surface_temperature_celcius': 59}}
{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-03-07T04:16:10', 'confidence': 88, 'surface_temperature_celcius': 64}}
{'air_temperature_celcius': 23, 'hotspot': {'datetime': '2017-03-09T13:23:40', 'confidence': 86, 'surface_temperature_celcius': 41}}
{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-03-10T04:48:40', 'confidence': 100, 'surface_temperature_celcius': 105}}
{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-03-10T04:46:20', 'confidence': 100, 'surface_temperature_celcius': 109}}
{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-03-10T04:45:30', 'confidence': 94, 'surface_temperature_celcius': 73}}
{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-03-

{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-04T04:32:50', 'confidence': 82, 'surface_temperature_celcius': 55}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-04T04:32:50', 'confidence': 100, 'surface_temperature_celcius': 123}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-04T04:32:50', 'confidence': 82, 'surface_temperature_celcius': 55}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-04T04:32:50', 'confidence': 100, 'surface_temperature_celcius': 118}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-04T04:32:40', 'confidence': 80, 'surface_temperature_celcius': 54}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-04T04:32:40', 'confidence': 96, 'surface_temperature_celcius': 78}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-04T04:32:40', 'confidence': 93, 'surface_temperature_celcius': 72}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-

{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-04-06T04:22:20', 'confidence': 83, 'surface_temperature_celcius': 56}}
{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-04-06T04:22:20', 'confidence': 81, 'surface_temperature_celcius': 71}}
{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-04-06T04:22:20', 'confidence': 85, 'surface_temperature_celcius': 60}}
{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-04-06T04:22:20', 'confidence': 100, 'surface_temperature_celcius': 103}}
{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-04-06T04:22:20', 'confidence': 91, 'surface_temperature_celcius': 69}}
{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-04-06T04:22:10', 'confidence': 86, 'surface_temperature_celcius': 68}}
{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-04-06T04:22:10', 'confidence': 85, 'surface_temperature_celcius': 59}}
{'air_temperature_celcius': 19, 'hotspot': {'datetime': '2017-04-06

{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-13T04:26:30', 'confidence': 100, 'surface_temperature_celcius': 111}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-13T04:26:30', 'confidence': 100, 'surface_temperature_celcius': 93}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-13T04:26:30', 'confidence': 95, 'surface_temperature_celcius': 77}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-13T04:26:30', 'confidence': 81, 'surface_temperature_celcius': 54}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-13T04:26:30', 'confidence': 95, 'surface_temperature_celcius': 75}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-13T04:26:30', 'confidence': 84, 'surface_temperature_celcius': 58}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-13T04:26:30', 'confidence': 84, 'surface_temperature_celcius': 58}}
{'air_temperature_celcius': 16, 'hotspot': {'datetime': '2017-04-1

In [16]:
print("records found:", i)

records found: 1133


In [17]:
hot_df[(80<=hot_df.confidence) & (hot_df.confidence<=100)].shape[0]

1133

> **e**. Find the top 10 records with the highest ​surface temperature​ ​(°C).

In [18]:
results = collection.aggregate([
    {"$unwind":"$hotspot"},
    {'$sort':{'hotspot.surface_temperature_celcius':-1}},
    {'$limit':10},
    {"$project":{"_id":0,"hotspot.surface_temperature_celcius":1}}
    
])
i = 0
for doc in results:
    i+=1
    print(doc['hotspot'])

{'surface_temperature_celcius': 124}
{'surface_temperature_celcius': 123}
{'surface_temperature_celcius': 122}
{'surface_temperature_celcius': 121}
{'surface_temperature_celcius': 120}
{'surface_temperature_celcius': 120}
{'surface_temperature_celcius': 120}
{'surface_temperature_celcius': 120}
{'surface_temperature_celcius': 120}
{'surface_temperature_celcius': 119}


In [19]:
hot_df["surface_temperature_celcius"].sort_values(ascending=False)[0:10]

1187    124
2381    123
1045    122
2620    121
2388    120
551     120
1727    120
1340    120
2324    120
875     119
Name: surface_temperature_celcius, dtype: int64

> **f**. Find the number of fire in each day. You are required to only display ​the total number of fire​ a​ nd ​the date​ in the output.

In [20]:
results = collection.aggregate([
    {"$unwind":"$hotspot"},
    {"$group":{'_id':'$date','count':{'$sum':1}}},
    {"$sort": {"_id":1}},
    
])
i = 0
for doc in results:
    i+=1
    print(doc)

{'_id': datetime.datetime(2017, 1, 4, 0, 0), 'count': 7}
{'_id': datetime.datetime(2017, 1, 5, 0, 0), 'count': 20}
{'_id': datetime.datetime(2017, 1, 6, 0, 0), 'count': 2}
{'_id': datetime.datetime(2017, 1, 7, 0, 0), 'count': 4}
{'_id': datetime.datetime(2017, 1, 8, 0, 0), 'count': 2}
{'_id': datetime.datetime(2017, 1, 10, 0, 0), 'count': 8}
{'_id': datetime.datetime(2017, 2, 4, 0, 0), 'count': 5}
{'_id': datetime.datetime(2017, 2, 5, 0, 0), 'count': 10}
{'_id': datetime.datetime(2017, 2, 6, 0, 0), 'count': 11}
{'_id': datetime.datetime(2017, 2, 7, 0, 0), 'count': 8}
{'_id': datetime.datetime(2017, 2, 8, 0, 0), 'count': 2}
{'_id': datetime.datetime(2017, 2, 10, 0, 0), 'count': 7}
{'_id': datetime.datetime(2017, 3, 4, 0, 0), 'count': 72}
{'_id': datetime.datetime(2017, 3, 5, 0, 0), 'count': 64}
{'_id': datetime.datetime(2017, 3, 6, 0, 0), 'count': 2}
{'_id': datetime.datetime(2017, 3, 10, 0, 0), 'count': 18}
{'_id': datetime.datetime(2017, 3, 13, 0, 0), 'count': 2}
{'_id': datetime.date

In [21]:
print("day to be counted:", i)

day to be counted: 145


In [22]:
len(clim_df.date)

366

In [23]:
len(hot_df.date.value_counts())

145

In [24]:
hot_df.date.value_counts().sort_index()[0:5]

2017-01-04     7
2017-01-05    20
2017-01-06     2
2017-01-07     4
2017-01-08     2
Name: date, dtype: int64

> **g**. Find the ​average surface temperature ​(°C) for each day​. You are required to only display ​average surface temperature (°C)​ ​and ​the date​ in the output.

In [25]:
results = collection.aggregate([
    {"$unwind":"$hotspot"},
    {'$group':{'_id':'$date','avg':{'$avg':"$hotspot.surface_temperature_celcius"}}},
    {'$sort':{'_id':1}}
    
])
i = 0
for doc in results:
    i+=1
    print(doc)

{'_id': datetime.datetime(2017, 1, 4, 0, 0), 'avg': 46.714285714285715}
{'_id': datetime.datetime(2017, 1, 5, 0, 0), 'avg': 68.4}
{'_id': datetime.datetime(2017, 1, 6, 0, 0), 'avg': 54.0}
{'_id': datetime.datetime(2017, 1, 7, 0, 0), 'avg': 30.5}
{'_id': datetime.datetime(2017, 1, 8, 0, 0), 'avg': 58.0}
{'_id': datetime.datetime(2017, 1, 10, 0, 0), 'avg': 48.25}
{'_id': datetime.datetime(2017, 2, 4, 0, 0), 'avg': 45.2}
{'_id': datetime.datetime(2017, 2, 5, 0, 0), 'avg': 55.6}
{'_id': datetime.datetime(2017, 2, 6, 0, 0), 'avg': 47.72727272727273}
{'_id': datetime.datetime(2017, 2, 7, 0, 0), 'avg': 43.5}
{'_id': datetime.datetime(2017, 2, 8, 0, 0), 'avg': 63.5}
{'_id': datetime.datetime(2017, 2, 10, 0, 0), 'avg': 43.57142857142857}
{'_id': datetime.datetime(2017, 3, 4, 0, 0), 'avg': 58.44444444444444}
{'_id': datetime.datetime(2017, 3, 5, 0, 0), 'avg': 56.796875}
{'_id': datetime.datetime(2017, 3, 6, 0, 0), 'avg': 47.0}
{'_id': datetime.datetime(2017, 3, 10, 0, 0), 'avg': 50.0}
{'_id': da

In [26]:
print("day to be counted:", i)

day to be counted: 145


In [27]:
len(hot_df.groupby(by="date").mean()["surface_temperature_celcius"])

145

In [28]:
hot_df.groupby(by="date").mean()["surface_temperature_celcius"][0:5]

date
2017-01-04    46.714286
2017-01-05    68.400000
2017-01-06    54.000000
2017-01-07    30.500000
2017-01-08    58.000000
Name: surface_temperature_celcius, dtype: float64